In [1]:
%matplotlib inline

import traceback
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic
from IPython.display import display, Audio, Image

# Python and numpy

* Organize Python modules
* Use numpy for array-oriented code
* Python tools for numerical analysis

# Organize Python modules

* One file containing Python code as a **script**
* One Python file is a “**module**”
* One directory containing Python files (satisfying some rules) is a “**package**”
* “Module” is usually used in a loose way to refer to things that may be imported by Python “import” statement.  Then a “module” can mean (the strictly defined) module or a package.

See examples in the [current directory](./)

# What is a script and how it works

* A script is a text file that the program loader sends to an engine (usually interpreter) to execute the content.
* Executable permission needs to be set for the shell to run it:

```
$ ls -al step0.py      # executable bit not set
-rw-r--r--  1 yungyuc  staff  574 Apr  7 22:18 step0.py
$ ./step0.py pstake.py # can't run without permission
-bash: ./step0.py: Permission denied
$ chmod a+x step0.py   # set the executable bit
$ ls -al step0.py      # executable bit is set
-rwxr-xr-x  1 yungyuc  staff  574 Apr  7 22:18 step0.py*
$ ./step0.py pstake.py # properly it runs
811 lines in pstake.py
```

# What’s a script

* Scripts usually are for automating repetitive work.
* Scripts should be short for quick implementation.

The leading line in a script that starts with `#!` is called the *shebang*.  It tells the program loader which executable to run for the script.

```python
#!/usr/bin/env python3
# ...
```

It won't work if executable permission isn't set on the script.

# Example: line counting (`step0`)

This is the first example: `step0.py` file in the [current directory](./).  It counts the number of lines in the file specified with the first argument.

In [2]:
!chmod u+x step0.py
!./step0.py pstake.py

811 lines in pstake.py


Another way, regardless the executable bit, to run the script is to explicitly call the Python executable.

In [3]:
!chmod u-x step0.py
!python3 step0.py pstake.py

811 lines in pstake.py


# One-liner

Python executable supports the <font color="red">**`-c`**</font> argument for one-liner.  The content of the script is passed from the command line.  It's called one-liner because it usually only takes one line.

One-liners are convenient for code that is only run once.  Quick to write but hard to read.

In [4]:
!python3 -c 'print(len(open("pstake.py").readlines()), "lines")'

811 lines


# Make a module

See the example file `step1.py` in the [current directory](./).  It factors out the line-counting code to a distinct function:

```python
def count_line(fname):
    if os.path.exists(fname):
        with open(fname) as fobj:
            lines = fobj.readlines()
        sys.stdout.write('{} lines in {}\n'.format(len(lines), fname))
    else:
        sys.stdout.write('{} not found\n'.format(fname))
```

The other code is for processing command-line arguments.  It's only useful for a script, so we move it into an `if` test:

```python
# This tests whether the code is evaluated as a script.
if __name__ == '__main__':
    if len(sys.argv) < 2:
        sys.stdout.write('missing file name\n')
    elif len(sys.argv) > 2:
        sys.stdout.write('only one argument is allowed\n')
    else:
        count_line(sys.argv[1])
```

# Different behaviors on import

Because **`step1`** checks for `__main__`, when it is imported as a module, nothing happens:

In [5]:
!python3 -c 'import step1'

But importing **`step0`** runs the code:

In [6]:
!python3 -c 'import step0' pstake.py

811 lines in pstake.py


To run the code, we now need to call the function defined in the **`step1`** module:

In [7]:
!python3 -c 'import step1 ; step1.count_line("pstake.py")'

811 lines in pstake.py


But when running as a script, both behave the same:

In [8]:
!python3 step0.py pstake.py

811 lines in pstake.py


In [9]:
!python3 step1.py pstake.py

811 lines in pstake.py


# Run module as script

Python executable supports the **`-m`** argument.  It imports the script as a module, and still runs it as a script.

In [10]:
!python3 -m step1 pstake.py

811 lines in pstake.py


With `python -m`, `step0.py` and `step1.py` again behave the same:

In [11]:
!python3 -m step0 pstake.py

811 lines in pstake.py


# Make the module more like a library

It's common to further factor out the code for script to a **`main`** function.  See the example file `step2.py` in the [current directory](./).

```python
def main():
    if len(sys.argv) < 2:
        sys.stdout.write('missing file name\n')
    elif len(sys.argv) > 2:
        sys.stdout.write('only one argument is allowed\n')
    else:
        count_line(sys.argv[1])


# This tests whether the file is evaluated as a script.
if __name__ == '__main__':
    main()
```

The behavior is the same as **`step1`**:

In [12]:
# run as a script
!python3 step2.py pstake.py

811 lines in pstake.py


In [13]:
# run the module as a script
!python3 -m step2 pstake.py

811 lines in pstake.py


In [14]:
# only import the module
!python3 -c 'import step2'

In [15]:
# import and then run the new main function
!python3 -c 'import step2 ; step2.main()' pstake.py

811 lines in pstake.py


# Make a package

When the code grows to a point, you may need a directory to house it.  Let's use our simple example to show how to make a package.  See the [step3 example directory](./step3/).

No file in the package version `step3` can be run as a script.

In [16]:
# The package __init__.py doesn't work like a module.
!python3 step3/__init__.py numpy.ipynb

Traceback (most recent call last):
  File "step3/__init__.py", line 11, in <module>
    from ._core import count_line
ImportError: attempted relative import with no known parent package


Everything else remains working, including the `-m` option of Python executable.

In [17]:
!python3 -m step3 numpy.ipynb

1479 lines in numpy.ipynb


In [18]:
!python3 -c 'import step3 ; step3.main()' numpy.ipynb

1479 lines in numpy.ipynb


# A really useful script

Here I show a real-world example (`pstake.py` in the [current directory](./)) for how to write a useful script: convert [pstricks](http://tug.org/PSTricks/main.cgi/) to an image file.

In [19]:
!rm -f cce.png
!ls cce.png
!./pstake.py cce.tex cce.png 2>&1 > /dev/null
!ls -al cce.png
Image(url="cce.png", width=800)

ls: cce.png: No such file or directory
-rw-r--r--  1 yungyuc  staff  21458 Sep 30 21:45 cce.png


# Numpy for array-centric code

* Arrays are the best tool to manage homogeneous data.
* The [numpy](http://www.numpy.org/) library provides everything we need for arrays in Python.
* Arrays use contigous memory, sequence doesn't.

In [20]:
# Make a list (one type of Python sequence) of integers.
lst = [1, 1, 2, 3, 5]
print('A list:', lst)

A list: [1, 1, 2, 3, 5]


In [21]:
# Import the numpy library. It's a universal convention to alias it to "np".
import numpy as np
# Make an array from the sequence.
array = np.array(lst)
print('An array:', np.array(array))

An array: [1 1 2 3 5]


# Key meta-data

In [22]:
array = np.array([[0, 1, 2], [3, 4, 5]])
print("shape:", array.shape)
print("size:", array.size)
print("nbytes:", array.nbytes)
print("itemsize:", array.itemsize)
print("dtype:", array.dtype)

shape: (2, 3)
size: 6
nbytes: 48
itemsize: 8
dtype: int64


# Data type

The numpy array is of type `numpy.ndarray`.  An `ndarray` has a property `dtype` for the data type the array uses:

In [23]:
print(type(array))
print(array.dtype)

<class 'numpy.ndarray'>
int64


`numpy.array` is the most basic `ndarray` constructor.  It detects the types in the input sequence data and choose the appropriate dtype for the constructed array.

In [24]:
array1 = np.array([1, 1, 2, 3, 5]) # only integer
print("only int:", array1, type(array1), array1.dtype)
array2 = np.array([1.0, 1.0, 2.0, 3.0, 5.0]) # only real
print("only real:", array2, type(array2), array2.dtype)
array3 = np.array([1, 1, 2, 3, 5.0]) # integer and real
print("int and real:", array3, type(array3), array3.dtype)

only int: [1 1 2 3 5] <class 'numpy.ndarray'> int64
only real: [1. 1. 2. 3. 5.] <class 'numpy.ndarray'> float64
int and real: [1. 1. 2. 3. 5.] <class 'numpy.ndarray'> float64


* A Python list doesn't know the type it contains, but an array does.
* The type information allows numpy to process the array data using pre-compiled C code.

# Construction

Numpy provides a lot of helpers to construct arrays ([see here](https://www.numpy.org/devdocs/reference/routines.array-creation.html)).  The 3 most common constructors are `empty`, `zeros`, and `ones`:

In [25]:
empty_array = np.empty(4)
print("It will contain garbage, but it doesn't waste time to initialize:", empty_array)
zeroed_array = np.zeros(4)
print("The contents are cleared with zeros:", zeroed_array)
unity_array = np.ones(4)
print("Instead of zeros, fill it with ones:", unity_array)
print("All of their data types are float64 (double-precision floating-point):",
      empty_array.dtype, zeroed_array.dtype, unity_array.dtype)

It will contain garbage, but it doesn't waste time to initialize: [5.e-324 5.e-324 5.e-324 0.e+000]
The contents are cleared with zeros: [0. 0. 0. 0.]
Instead of zeros, fill it with ones: [1. 1. 1. 1.]
All of their data types are float64 (double-precision floating-point): float64 float64 float64


* `full` is a shorthand for `empty` and `fill`:

In [26]:
empty_array = np.empty(4)
empty_array.fill(7)
print("It's the same as creating an empty array and fill the value:", empty_array)
filled_array = np.full(4, 7)
print("Build an array populated with an arbitrary value:", filled_array)
filled_real_array = np.full(4, 7.0)
print("Build an array populated with an arbitrary real value:", filled_real_array)

It's the same as creating an empty array and fill the value: [7. 7. 7. 7.]
Build an array populated with an arbitrary value: [7 7 7 7]
Build an array populated with an arbitrary real value: [7. 7. 7. 7.]


* `arange` builds a monotonically increasing array:

In [27]:
ranged_array = np.arange(4)
print("Build an array with range:", ranged_array)
ranged_real_array = np.arange(4.0)
print("Build with real range:", ranged_real_array)

Build an array with range: [0 1 2 3]
Build with real range: [0. 1. 2. 3.]


`linspace` returns an array whose elements are evenly placed in a closed interval:

In [28]:
linear_array = np.linspace(11, 13, num=6)
print("Create an equally-spaced array with 6 elements:", linear_array)

Create an equally-spaced array with 6 elements: [11.  11.4 11.8 12.2 12.6 13. ]


# Multi-dimensional arrays

* Multi-dimensional arrays are the building-block of matrices and linear algebra.  Much more useful than one-dimensional arrays.
* Create multi-dimensional arrays by stacking 1D:

In [29]:
ranged_array = np.arange(10)
print("A 1D array:", ranged_array)
hstack_array = np.hstack([ranged_array, ranged_array])
print("Horizontally stacked array:", hstack_array)
vstack_array = np.vstack([ranged_array, ranged_array])
print("Vertically stacked array:", vstack_array)

A 1D array: [0 1 2 3 4 5 6 7 8 9]
Horizontally stacked array: [0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9]
Vertically stacked array: [[0 1 2 3 4 5 6 7 8 9]
 [0 1 2 3 4 5 6 7 8 9]]


* `ndarray` by default is row-majoring:

\begin{align*}
A = \left(\begin{array}{ccc}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
0 & 1 & 2 \\
3 & 4 & 5
\end{array}\right)
\end{align*}

In [30]:
original_array = np.arange(6)
print("original 1D array:", original_array)

original 1D array: [0 1 2 3 4 5]


In [31]:
print("reshaped 2D array:", original_array.reshape((2,3)))

reshaped 2D array: [[0 1 2]
 [3 4 5]]


* Column-majoring:

In [32]:
print("reshaped 2D array:", original_array.reshape((2,3), order='f'))

reshaped 2D array: [[0 2 4]
 [1 3 5]]


Example for 3D arrays:

In [33]:
original_array = np.arange(24)
print("original 1D array:", original_array)

original 1D array: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [34]:
reshaped_array = original_array.reshape((2,3,4))
print("reshaped 3D array:", reshaped_array)

reshaped 3D array: [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


For multi-dimensional arrays, operations can be done along any of the axes.  Take 2D for example.  First sum along the 0th-axis:

\begin{align*}
A_{\mathrm{along } 0} = \left(\begin{array}{ccc}
a_{00} + a_{10} & a_{01} + a_{11} & a_{02} + a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
3 & 5 & 7
\end{array}\right)
\end{align*}

Do it along the 1st-axis:

\begin{align*}
A_{\mathrm{along } 1} = \left(\begin{array}{cc}
a_{00} + a_{01} + a_{02} & a_{10} + a_{11} + a_{12}
\end{array}\right)
= \left(\begin{array}{ccc}
3 & 12
\end{array}\right)
\end{align*}

In [35]:
print("Summation along 0th axis:", reshaped_array.sum(axis=0))

Summation along 0th axis: [[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]


In [36]:
print("Summation along 1st axis:", reshaped_array.sum(axis=1))

Summation along 1st axis: [[12 15 18 21]
 [48 51 54 57]]


# Selection: extract sub-array

There are 3 ways to create sub-arrays:
1. Slicing
2. Integer indexing
3. Boolean indexing

# Slicing

The array created from slicing shares the buffer of the original one:

In [37]:
array = np.arange(10)
print("This is the original array:", array)

sub_array = array[:5]
print("This is the sub-array:", sub_array)

sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed:", sub_array)

print("And the original array is changed too (!):", array)

This is the original array: [0 1 2 3 4 5 6 7 8 9]
This is the sub-array: [0 1 2 3 4]
The sub-array is changed: [4 3 2 1 0]
And the original array is changed too (!): [4 3 2 1 0 5 6 7 8 9]


New buffer can be created by copying the returned array:

In [38]:
array = np.arange(10.0)
print("Recreate the original array to show how to avoid this:", array)

# Make a copy from the slice.
sub_array = array[:5].copy()
sub_array[:] = np.arange(4, -1, -1)
print("The sub-array is changed, again:", sub_array)
print("But original array remains the same:", array)

Recreate the original array to show how to avoid this: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
The sub-array is changed, again: [4. 3. 2. 1. 0.]
But original array remains the same: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


Slice one dimension in a multi-dimensional array:

In [39]:
array = np.arange(24).reshape((2,3,4))
print("orignal:\n%s" % array)
array[:,1,3] = np.arange(300,302)
print("find 300, 301:\n%s" % array)

orignal:
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
find 300, 301:
[[[  0   1   2   3]
  [  4   5   6 300]
  [  8   9  10  11]]

 [[ 12  13  14  15]
  [ 16  17  18 301]
  [ 20  21  22  23]]]


Slice two dimensions in a multi-dimensional array:

In [40]:
array = np.arange(24).reshape((2,3,4))
print("orignal:\n%s" % array)
array[:,0,:] = np.arange(200,208).reshape((2,4))
print("find the number [200,208):\n%s" % array)

orignal:
[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
find the number [200,208):
[[[200 201 202 203]
  [  4   5   6   7]
  [  8   9  10  11]]

 [[204 205 206 207]
  [ 16  17  18  19]
  [ 20  21  22  23]]]


# Integer indexing

In [41]:
array = np.arange(100, 106)
slct = np.array([1, 3])
print("select by indice 1, 3:", array[slct])
slct = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5])
print("new array is bigger than the old one:", array[slct])
array2 = array.reshape((2,3))
slct = np.array([1])
print("select by indice 1:", array2[slct])
slct = np.array([[0,0], [0,1], [1,2]])
print("select by indice (0,0), (0,1), (1,2):", array2[slct[:,0],slct[:,1]],
      "using", slct)

select by indice 1, 3: [101 103]
new array is bigger than the old one: [100 100 101 101 102 102 103 103 104 104 105 105]
select by indice 1: [[103 104 105]]
select by indice (0,0), (0,1), (1,2): [100 101 105] using [[0 0]
 [0 1]
 [1 2]]


# Boolean selection

The Boolean arrays filter wanted or unwanted elements in another array.

In [42]:
less_than_5 = ranged_array < 5
print("The mask for less than 5:", less_than_5)
print("The values that are less than 5", ranged_array[less_than_5])

all_on_mask = np.ones(10, dtype='bool')
print("All on mask:", all_on_mask)

all_off_mask = np.zeros(10, dtype='bool')
print("All off mask:", all_off_mask)

The mask for less than 5: [ True  True  True  True  True False False False False False]
The values that are less than 5 [0 1 2 3 4]
All on mask: [ True  True  True  True  True  True  True  True  True  True]
All off mask: [False False False False False False False False False False]


# Broadcasting

[Broadcasting](https://docs.scipy.org/doc/numpy-1.16.1/reference/ufuncs.html#broadcasting) handles arrays of different shapes participating in an operation.

1. All input arrays with number of dimension smaller than the input array of largest number of dimension, have 1’s prepended to their shapes.
2. The size in each dimension of the output shape is the maximum of all the input sizes in that dimension.
3. An input can be used in the calculation if its size in a particular dimension either matches the output size in that dimension, or has value exactly 1.
4. If an input has a dimension size of 1 in its shape, the first data entry in that dimension will be used for all calculations along that dimension.

Broadcasting is to handle arrays of different shapes.

In [43]:
a = np.arange(2); print("a =", a)
b = np.arange(10,12); print("b =", b)
print("a+b =", a+b) # good: same shape
c = np.arange(3); print("c =", c)
try:
    print(a+c)
except ValueError:
    print("cannot do a+c: a.shape %s != c.shape %s" % (a.shape, c.shape))

a = [0 1]
b = [10 11]
a+b = [10 12]
c = [0 1 2]
cannot do a+c: a.shape (2,) != c.shape (3,)


In [44]:
a = np.arange(5,7).reshape((2,1))
b = np.arange(10,13).reshape((1,3))
c = np.arange(100,103)
d = np.array(1000)
print(a, b, c, d)
print(a.shape, b.shape, c.shape, d.shape)
v = np.arange(2*3).reshape((2,3))
print(v, v.shape)
print(a*b)

[[5]
 [6]] [[10 11 12]] [100 101 102] 1000
(2, 1) (1, 3) (3,) ()
[[0 1 2]
 [3 4 5]] (2, 3)
[[50 55 60]
 [60 66 72]]


# Python tools for numerical analysis

There are two equally important activities for software development.  One is to write code.  We will need to learn some basic concepts to write meaningful code.

The other is to use code written by other people.  Especially in the early stage of development, we want to quickly see the results.  We may just use the results of other software.  We may directly incoporate the foreign (usually, also called "third-party") software, if the situation allows.  Otherwise, we can replace the quick prototype in a later phase.

In this lecture, I will introduce 4 useful tools for numerical analysis that you may use thoughout the course and your future work.

# Jupyter (notebook)

"Project Jupyter exists to develop open-source software, open-standards, and services for interactive computing across dozens of programming languages." -- https://jupyter.org

* This presentation is done by using Jupyter
* Show the code and run it in the same time
* Terminal access
* File management
* JupyterLab integrated environment

Jupyter is a client-server system.  What we are touching and playing is its "frontend", the interactive user interface.  It talks to the "backend", which is called a Jupyter kernel.  See the following image ([source](https://jupyter.readthedocs.io/en/latest/architecture/how_jupyter_ipython_work.html)):

<center><img src="https://jupyter.readthedocs.io/en/latest/_images/notebook_components.png" alt="Jupyter distributed architecture" /></center>

The system is distributed.  The browser and the Jupyter server run on different computers and HTTP is used to connect them.  The kernel can also be configured to run on a different computer than the server.

Jupyter has 3 types of cells:

1. Code.  The content will be executed.
2. [Markdown](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#markdown-cells).  Use a mark-up language called "[markdown](https://daringfireball.net/projects/markdown/)" to format text.
3. Raw nbconvert.  Jupyter skip processing the content and pass is through to other converting tools.

Most of the time we only care about the interactive computing capability provided by the code cell.

# Python code

In [45]:
import numpy as np
v1 = np.array([1,1,1], dtype='float64')
v2 = np.array([1,-1,0], dtype='float64')
print("dot product between v1 and v2:", v1.dot(v2))
print("v1 length:", np.sqrt((v1**2).sum()))

dot product between v1 and v2: 0.0
v1 length: 1.7320508075688772


In [46]:
# simple math
d = 30.*np.pi/180.
print("trigonometric function at 30 degree:", np.sin(d), np.cos(d), np.tan(d))

trigonometric function at 30 degree: 0.49999999999999994 0.8660254037844387 0.5773502691896256


# IPython magic

[IPython](https://ipython.readthedocs.io/) provides the Jupyter kernel for enhanced interactive execution.  The "magic" are part of the enhancements.  There are two types of magic commands: line and cell.  A line magic is a line starting with "`%`".

In [47]:
import sys
print(sys.path) # show python import paths
%pwd # print current directory

['/Users/yungyuc/hack/code/nsd/notebook/numpy', '/Users/yungyuc/hack/usr/opt37_190418/lib/python37.zip', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/lib-dynload', '', '/Users/yungyuc/.local/lib/python3.7/site-packages', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/Cython-0.28.3-py3.7-macosx-10.14-x86_64.egg', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/pybind11-2.3.dev0-py3.7.egg', '/Users/yungyuc/hack/usr/opt37_190418/lib/python3.7/site-packages/IPython/extensions', '/Users/yungyuc/.ipython']


'/Users/yungyuc/hack/code/nsd/notebook/numpy'

# Cell magic

A line starting with "`%%`" indicates a magic that takes all the content of a cell.

In [48]:
%%script bash
whoami
pwd
ls -l

yungyuc
/Users/yungyuc/hack/code/nsd/notebook/numpy
total 232
drwxr-xr-x  8 yungyuc  staff    256 Sep 30 21:45 __pycache__
-rw-r--r--  1 yungyuc  staff  21458 Sep 30 21:45 cce.png
-rw-r--r--  1 yungyuc  staff    828 Jun  2 17:55 cce.tex
-rw-r--r--  1 yungyuc  staff  37375 Sep 28 18:11 numpy.ipynb
-rwxr-xr-x  1 yungyuc  staff  29628 Jun  2 17:55 pstake.py
-rw-r--r--  1 yungyuc  staff    574 Sep 30 17:43 step0.py
-rwxr-xr-x  1 yungyuc  staff    804 Sep 24 16:30 step0_hw3.py
-rw-r--r--  1 yungyuc  staff    961 Jun  2 17:55 step1.py
-rw-r--r--  1 yungyuc  staff    917 Jun  2 17:55 step2.py
drwxr-xr-x  6 yungyuc  staff    192 Jun  2 17:55 step3


# Other features

* Escape to shell in a line starting with "`!`":

In [49]:
!uptime

21:45  up 26 days, 14:36, 11 users, load averages: 2.50 1.36 1.32


* Editor
* Terminal

# Drawing using matplotlib

[Matplotlib](https://matplotlib.org) is a powerful library for 2D plotting.  It can be used standalone or integrated with Jupyter notebook.  The following magic enables the integration.

In [50]:
%matplotlib inline

# Linear algebra with numpy and scipy

Use a singular value problem to demonstrate how to use the linear algebra tools provided in numpy and scipy.

# Package managers

To study this course you would need a runtime environment that has the software tools installed.  Although manually building all the dependencies from source is sometimes unavoidable, it's too time-consuming to do it always.

Usually we will use a package manager to help.  A package manager provides recipes for building package from source, and also pre-built binary packages.  It defines the dependencies between the packages.  For example, for scipy to work, numpy needs to be installed beforehand.  A package manager should allow automatic installation of numpy when you request scipy.

In the numerical analysis world, [conda](https://conda.io/) is one of the most versatile package manager that we will use.

# conda

* Anaconda
* Conda-forge

# pip

"[pip](https://pip.pypa.io/) is the package installer for Python.  You can use pip to install packages from the [Python Package Index](https://pypi.org/) and other indexes."

# Exercises

1. List all primitive types supported by `numpy.ndarray` on x86-64.
2. Port "`step0.py`" to use bash.
3. Modify the script "`step0.py`" so that it reads the environment variable named "`PYTHON_BIN`" that specifies the location of the Python executable for the script.  Hint: play a trick (or tricks) using bash, and note it's possible to write no-op command in bash.

# References

* [Broadcasting arrays in Numpy](https://eli.thegreenplace.net/2015/broadcasting-arrays-in-numpy/) by Eli Bendersky